In [1]:
import torch
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
train_data = datasets.CIFAR10(
    root = 'data',
    train = True,                         
    transform = transforms.Compose([ToTensor()]), 
    download = False,            
)
test_data = datasets.CIFAR10(
    root = 'data', 
    train = False, 
    transform = transforms.Compose([ToTensor()]), 
)

In [2]:
print(train_data)


Dataset CIFAR10
    Number of datapoints: 50000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )


In [3]:
from torch.utils.data import DataLoader
loaders = {
    'train' : DataLoader(train_data, 
                         batch_size=100, 
                         shuffle=True, 
                         num_workers=1),
    
    'test'  : DataLoader(test_data, 
                         batch_size=100, 
                         shuffle=True, 
                         num_workers=1),
}
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f91789717f0>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7f9178971880>}

In [4]:
import torch.nn as nn
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(3, 16, 5, 1, 2),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # fully connected layer, output 10 classes
        self.out = nn.Linear(32 * 8 * 8, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output
    
    def extract_conv_features(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        return x


In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [6]:
cnn = CNN()


In [7]:
cnn.to(device)

CNN(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=2048, out_features=10, bias=True)
)

In [8]:
loss_func = nn.CrossEntropyLoss()   


In [9]:
from torch import optim
optimizer = optim.Adam(cnn.parameters(), lr = 0.01)   

In [10]:
def test():
    # Test the model
    cnn.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in loaders['test']:
            images, labels = images.to(device), labels.to(device)
            test_output = cnn(images)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))  
            pass
    return accuracy
#         print('Test Accuracy of the model on the 10000 test images: %.2f' % accuracy)
#     pass

In [13]:
from torch.autograd import Variable

def train(num_epochs, cnn, loaders):
    
    cnn.train()
    total_step = len(loaders['train'])
    acc = 0
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            images, labels = images.to(device), labels.to(device)
            # gives batch data, normalize x when iterate train_loader
            b_x = Variable(images)   # batch x
            b_y = Variable(labels)   # batch y

            output = cnn(b_x)             
            loss = loss_func(output, b_y)
            
            # clear gradients for this training step   
            optimizer.zero_grad()           
            
            # backpropagation, compute gradients 
            loss.backward()    
            # apply gradients             
            optimizer.step()                
            if (i+1) % 100 == 0:
                new_acc = test()
                if new_acc > acc: 
                    torch.save(cnn, 'cifar10_cnn.pt')
                    acc = new_acc
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

In [14]:
num_epochs = 100
train(num_epochs, cnn, loaders)

Epoch [1/100], Step [100/500], Loss: 1.6021
Epoch [1/100], Step [200/500], Loss: 1.5380
Epoch [1/100], Step [300/500], Loss: 1.5379
Epoch [1/100], Step [400/500], Loss: 1.4380
Epoch [1/100], Step [500/500], Loss: 1.5712
Epoch [2/100], Step [100/500], Loss: 1.5189
Epoch [2/100], Step [200/500], Loss: 1.3923
Epoch [2/100], Step [300/500], Loss: 1.6453
Epoch [2/100], Step [400/500], Loss: 1.6479
Epoch [2/100], Step [500/500], Loss: 1.5492
Epoch [3/100], Step [100/500], Loss: 1.6752
Epoch [3/100], Step [200/500], Loss: 1.4033
Epoch [3/100], Step [300/500], Loss: 1.3425
Epoch [3/100], Step [400/500], Loss: 1.5029
Epoch [3/100], Step [500/500], Loss: 1.5079
Epoch [4/100], Step [100/500], Loss: 1.6126
Epoch [4/100], Step [200/500], Loss: 1.2708
Epoch [4/100], Step [300/500], Loss: 1.5498
Epoch [4/100], Step [400/500], Loss: 1.4909
Epoch [4/100], Step [500/500], Loss: 1.4609
Epoch [5/100], Step [100/500], Loss: 1.4067
Epoch [5/100], Step [200/500], Loss: 1.4664
Epoch [5/100], Step [300/500], L

Epoch [37/100], Step [500/500], Loss: 1.3715
Epoch [38/100], Step [100/500], Loss: 1.3997
Epoch [38/100], Step [200/500], Loss: 1.4164
Epoch [38/100], Step [300/500], Loss: 1.3959
Epoch [38/100], Step [400/500], Loss: 1.4077
Epoch [38/100], Step [500/500], Loss: 1.2927
Epoch [39/100], Step [100/500], Loss: 1.5284
Epoch [39/100], Step [200/500], Loss: 1.5895
Epoch [39/100], Step [300/500], Loss: 1.4101
Epoch [39/100], Step [400/500], Loss: 1.3592
Epoch [39/100], Step [500/500], Loss: 1.6519
Epoch [40/100], Step [100/500], Loss: 1.4872
Epoch [40/100], Step [200/500], Loss: 1.4891
Epoch [40/100], Step [300/500], Loss: 1.2751
Epoch [40/100], Step [400/500], Loss: 1.6873
Epoch [40/100], Step [500/500], Loss: 1.4296
Epoch [41/100], Step [100/500], Loss: 1.3567
Epoch [41/100], Step [200/500], Loss: 1.3459
Epoch [41/100], Step [300/500], Loss: 1.3469
Epoch [41/100], Step [400/500], Loss: 1.7197
Epoch [41/100], Step [500/500], Loss: 1.8063
Epoch [42/100], Step [100/500], Loss: 1.2648
Epoch [42/

Epoch [74/100], Step [300/500], Loss: 1.7009
Epoch [74/100], Step [400/500], Loss: 1.7163
Epoch [74/100], Step [500/500], Loss: 1.3505
Epoch [75/100], Step [100/500], Loss: 1.2915
Epoch [75/100], Step [200/500], Loss: 1.7081
Epoch [75/100], Step [300/500], Loss: 1.2387
Epoch [75/100], Step [400/500], Loss: 1.5188
Epoch [75/100], Step [500/500], Loss: 1.3098
Epoch [76/100], Step [100/500], Loss: 1.3547
Epoch [76/100], Step [200/500], Loss: 1.3430
Epoch [76/100], Step [300/500], Loss: 1.1752
Epoch [76/100], Step [400/500], Loss: 1.3917
Epoch [76/100], Step [500/500], Loss: 1.2255
Epoch [77/100], Step [100/500], Loss: 1.5792
Epoch [77/100], Step [200/500], Loss: 1.4802
Epoch [77/100], Step [300/500], Loss: 1.4854
Epoch [77/100], Step [400/500], Loss: 1.2029
Epoch [77/100], Step [500/500], Loss: 1.2724
Epoch [78/100], Step [100/500], Loss: 1.2239
Epoch [78/100], Step [200/500], Loss: 1.3227
Epoch [78/100], Step [300/500], Loss: 1.3961
Epoch [78/100], Step [400/500], Loss: 1.3601
Epoch [78/

In [15]:
test()

0.43

In [16]:
model = torch.load('cifar10_cnn.pt')
model.eval()

CNN(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=2048, out_features=10, bias=True)
)

In [18]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in loaders['test']:
        images, labels = images.to(device), labels.to(device)
        test_output = model(images)
        pred_y = torch.max(test_output, 1)[1].data.squeeze()
        accuracy = (pred_y == labels).sum().item() / float(labels.size(0))  
        pass
# return accuracy
    print('Test Accuracy of the model on the 10000 test images: %.2f' % accuracy)
#     pass

Test Accuracy of the model on the 10000 test images: 0.54
